In [1]:
import pickle
import logging
 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
 


In [2]:
import os
os.getcwd()

'/content'

In [6]:
!git clone https://github.com/dongwookim-ml/python-topic-model.git

Cloning into 'python-topic-model'...
remote: Enumerating objects: 357, done.
remote: Total 357 (delta 0), reused 0 (delta 0), pack-reused 357
Receiving objects: 100% (357/357), 18.75 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [ ]:
import ptm

In [ ]:
os.listdir()

['.config', 'python-topic-model', 'drive', 'sample_data']

In [ ]:
os.listdir('python-topic-model')

['.git',
 'LICENSE',
 'setup.py',
 'data',
 'notebook',
 '.gitignore',
 'ptm',
 'README.md']

In [ ]:
from pythontopicmodel import ptm

In [ ]:
from pythontopicmodel.ptm import RelationalTopicModel
from pythontopicmodel.ptm.utils import convert_cnt_to_list, get_top_words

In [ ]:
#     def fit(self, doc_ids, doc_cnt, doc_links, max_iter=100):
#         for di in xrange(self.n_doc):
#             unique_word = len(doc_ids[di])
#             cnt = doc_cnt[di]
#             self.phi.append(np.random.dirichlet([10] * self.n_topic, unique_word).T)  # list of KxW
#             self.pi[di, :] = np.sum(cnt * self.phi[di], 1) / np.sum(cnt * self.phi[di])

#         for iter in xrange(max_iter):
#             tic = time.time()
#             self.variation_update(doc_ids, doc_cnt, doc_links)
#             self.parameter_estimation(doc_links)
#             if self.verbose:
#                 elbo = self.compute_elbo(doc_ids, doc_cnt, doc_links)
#                 logger.info('[ITER] %3d,\tElapsed time: %.3f\tELBO: %.3f', iter, time.time()-tic, elbo)

In [ ]:
#     def compute_elbo(self, doc_ids, doc_cnt, doc_links):
#         """ compute evidence lower bound for trained model
#         """
#         elbo = 0

#         e_log_theta = psi(self.gamma) - psi(np.sum(self.gamma, 1))[:, np.newaxis]  # D x K
#         log_beta = np.log(self.beta + eps)

#         for di in xrange(self.n_doc):
#             words = doc_ids[di]
#             cnt = doc_cnt[di]

#             elbo += np.sum(cnt * (self.phi[di] * log_beta[:, words]))  # E_q[log p(w_{d,n}|\beta,z_{d,n})]
#             elbo += np.sum((self.alpha - 1.) * e_log_theta[di, :])  # E_q[log p(\theta_d | alpha)]
#             elbo += np.sum(self.phi[di].T * e_log_theta[di, :])  # E_q[log p(z_{d,n}|\theta_d)]

#             elbo += -gammaln(np.sum(self.gamma[di, :])) + np.sum(gammaln(self.gamma[di, :])) \
#                     - np.sum((self.gamma[di, :] - 1.) * (e_log_theta[di, :]))  # - E_q[log q(theta|gamma)]
#             elbo += - np.sum(cnt * self.phi[di] * np.log(self.phi[di]))  # - E_q[log q(z|phi)]

#             for adi in doc_links[di]:
#                 elbo += np.dot(self.eta,
#                                self.pi[di] * self.pi[adi]) + self.nu  # E_q[log p(y_{d1,d2}|z_{d1},z_{d2},\eta,\nu)]

#         return elbo

In [ ]:
#     def variation_update(self, doc_ids, doc_cnt, doc_links):
#         # update phi, gamma
#         e_log_theta = psi(self.gamma) - psi(np.sum(self.gamma, 1))[:, np.newaxis]

#         new_beta = np.zeros([self.n_topic, self.n_voca])

#         for di in xrange(self.n_doc):
#             words = doc_ids[di]
#             cnt = doc_cnt[di]
#             doc_len = np.sum(cnt)

#             new_phi = np.log(self.beta[:, words] + eps) + e_log_theta[di, :][:, np.newaxis]

#             gradient = np.zeros(self.n_topic)
#             for ai in doc_links[di]:
#                 gradient += self.eta * self.pi[ai, :] / doc_len

#             new_phi += gradient[:, np.newaxis]
#             new_phi = np.exp(new_phi)
#             new_phi = new_phi / np.sum(new_phi, 0)

#             self.phi[di] = new_phi

#             self.pi[di, :] = np.sum(cnt * self.phi[di], 1) / np.sum(cnt * self.phi[di])
#             self.gamma[di, :] = np.sum(cnt * self.phi[di], 1) + self.alpha
#             new_beta[:, words] += (cnt * self.phi[di])

#         self.beta = new_beta / np.sum(new_beta, 1)[:, np.newaxis]


In [ ]:
#     def parameter_estimation(self, doc_links):
#         # update eta, nu
#         pi_sum = np.zeros(self.n_topic)

#         num_links = 0.

#         for di in xrange(self.n_doc):
#             for adi in doc_links[di]:
#                 pi_sum += self.pi[di, :] * self.pi[adi, :]
#                 num_links += 1

#         num_links /= 2.  # divide by 2 for bidirectional edge
#         pi_sum /= 2.

#         pi_alpha = np.zeros(self.n_topic) + self.alpha / (self.alpha * self.n_topic) * self.alpha / (self.alpha * self.n_topic)

#         self.nu = np.log(num_links - np.sum(pi_sum)) - np.log(
#             self.rho * (self.n_topic - 1) / self.n_topic + num_links - np.sum(pi_sum))
#         self.eta = np.log(pi_sum) - np.log(pi_sum + self.rho * pi_alpha) - self.nu

In [ ]:
#     def save_model(self, output_directory, vocab=None):
#         import os
#         if not os.path.exists(output_directory):
#             os.mkdir(output_directory)

#         np.savetxt(output_directory + '/eta.txt', self.eta, delimiter='\t')
#         np.savetxt(output_directory + '/beta.txt', self.beta, delimiter='\t')
#         np.savetxt(output_directory + '/gamma.txt', self.gamma, delimiter='\t')
#         with open(output_directory + '/nu.txt', 'w') as f:
#             f.write('%f\n' % self.nu)

#         if vocab is not None:
#             write_top_words(self.beta, vocab, output_directory + '/top_words.csv')


In [ ]:
logger = logging.getLogger('RelationalTopicModel')
logger.propagate=False

%matplotlib inline

In [9]:
doc_ids = pickle.load(open('/content/python-topic-model/data/cora/doc_ids.pkl', 'rb'))
doc_cnt = pickle.load(open('/content/python-topic-model/data/cora/doc_cnt.pkl', 'rb'))
doc_links = pickle.load(open('/content/python-topic-model/data/cora/doc_links_sym.pkl', 'rb'))
voca = pickle.load(open('/content/python-topic-model/data/cora/voca.pkl', 'rb'))

In [14]:
len(doc_links)

13147

In [ ]:
n_doc = len(doc_ids)
n_topic = 10
n_voca = len(voca)
max_iter = 50

In [ ]:
model = RelationalTopicModel(n_topic, n_doc, n_voca, verbose=True)
model.fit(doc_ids, doc_cnt, doc_links, max_iter=max_iter)

2020-11-08 02:52:13 INFO:RelationalTopicModel:Initialize RTM: num_voca:17059, num_topic:10, num_doc:13147
2020-11-08 02:52:18 INFO:RelationalTopicModel:[ITER]   0,	Elapsed time: 4.510	ELBO: -7558288.201
2020-11-08 02:52:23 INFO:RelationalTopicModel:[ITER]   1,	Elapsed time: 4.514	ELBO: -7556048.516
2020-11-08 02:52:27 INFO:RelationalTopicModel:[ITER]   2,	Elapsed time: 4.492	ELBO: -7554126.601
2020-11-08 02:52:32 INFO:RelationalTopicModel:[ITER]   3,	Elapsed time: 4.527	ELBO: -7551822.870
2020-11-08 02:52:37 INFO:RelationalTopicModel:[ITER]   4,	Elapsed time: 4.506	ELBO: -7548381.234
2020-11-08 02:52:41 INFO:RelationalTopicModel:[ITER]   5,	Elapsed time: 4.498	ELBO: -7543005.584
2020-11-08 02:52:46 INFO:RelationalTopicModel:[ITER]   6,	Elapsed time: 4.500	ELBO: -7535108.203
2020-11-08 02:52:50 INFO:RelationalTopicModel:[ITER]   7,	Elapsed time: 4.454	ELBO: -7524580.960
2020-11-08 02:52:55 INFO:RelationalTopicModel:[ITER]   8,	Elapsed time: 4.581	ELBO: -7511770.503
2020-11-08 02:52:59 I

In [ ]:
for k in range(n_topic):
    top_words = get_top_words(model.beta, voca, k, 10)
    print('Topic', k, ':', ','.join(top_words))

Topic 0 : algorithm,time,problem,graph,tree,show,bound,result,class,network
Topic 1 : model,system,simulation,time,paper,performance,using,speech,result,dynamic
Topic 2 : image,robot,object,system,research,model,control,paper,part,motion
Topic 3 : system,language,design,software,agent,paper,application,model,information,user
Topic 4 : constraint,problem,paper,research,method,type,theory,science,document,reasoning
Topic 5 : learning,algorithm,network,problem,method,approach,rule,paper,data,task
Topic 6 : performance,data,parallel,memory,system,program,application,communication,paper,network
Topic 7 : query,program,system,database,logic,model,language,technique,data,paper
Topic 8 : algorithm,problem,method,solution,function,result,paper,matrix,using,space
Topic 9 : network,protocol,service,application,control,system,resource,packet,paper,traffic
